In [1]:
import arff
with open('vote.arff', 'r') as f:
    data = arff.load(f)

In [2]:
#load data into a list
datalist=[]
for i in data['data']:
    list=[]
    for j in range(len(i)):
        list.append(data['attributes'][j][0] + "=" + i[j])
    datalist.append(list)

In [3]:
min_supp = 0.5
min_conf = 1.0

In [4]:
def apriori(min_supp, min_conf):
    # initialize candidate itemsets of size 1
    C = []
    for transaction in datalist:
        for item in transaction:
            if [item] not in C:
                C.append([item])

    k=1
    allFrequent = []
    allCounts = {}
    
    print('Apriori\n=======\n\nGenerated sets of large itemsets:\n')
    
    while(True):
        frequent = []
        #get counts of each candidate itemset
        counts = {}
        for transaction in datalist:
            for itemset in C:
                if set(itemset).issubset(set(transaction)):
                    counts[tuple(itemset)] = counts.get(tuple(itemset), 0) + 1

        #eliminate the candidates without min_supp to get the frequent itemsets
        for itemset, count in counts.items():
            if count >= min_supp*len(datalist):
                allCounts[tuple(itemset)] = counts[tuple(itemset)]
                frequent.append(itemset)
                allFrequent.append(itemset)
        if not frequent:
            break #break out when the frequent itemlist is empty
        #generate new candidates from previous frequent itemsets
        C = []
        for i in range(len(frequent)):
            for j in range(i+1, len(frequent)):
                #only add new candidates if they include the frequent itemsets
                if frequent[i][:-1] == frequent[j][:-1]:
                    new_itemset = frequent[i] + (frequent[j][-1],)
                    C.append(new_itemset)
        print('Large itemsets L('+str(k)+'): ' + str(len(frequent)))
        print(frequent)
        print()
        k+=1
        
    return allFrequent, allCounts


In [5]:
def associations(allFrequent, allCounts):
    #get counts for all association combinations
    rule_num = 0
    for freq_itemset1 in allFrequent:
        for freq_itemset2 in allFrequent:
            combo = freq_itemset1 + freq_itemset2
            if combo == tuple([x for i, x in enumerate(combo) if x not in combo[:i]]): #ignore redundant combinations
                assoc = combo
                assoc_counts = 0
                for transaction in datalist:
                    if set(assoc).issubset(set(transaction)):
                        assoc_counts += 1

                #eliminate resulting associations using min_conf
                if assoc_counts / allCounts[freq_itemset1] >= min_conf:
                    rule_num+=1
                    print('Rule ' + str(rule_num) + ': '
                        + ' '.join(freq_itemset1) + ' ' + str(allCounts[freq_itemset1]) + ' ==> '
                        + ' '.join(freq_itemset2) + ' ' + str(allCounts[freq_itemset1])
                        + ' <conf: ' + str(assoc_counts/allCounts[freq_itemset1])
                        + '> <supp: ' + str(allCounts[freq_itemset1]/len(datalist)) + '>')


In [6]:
allFrequent, allCounts = apriori(min_supp, min_conf)
associations(allFrequent, allCounts)

Apriori

Generated sets of large itemsets:

Large itemsets L(1): 17
[('handicapped-infants=n',), ('water-project-cost-sharing=y',), ('el-salvador-aid=y',), ('religious-groups-in-schools=y',), ('immigration=y',), ('synfuels-corporation-cutback=n',), ('superfund-right-to-sue=y',), ('crime=y',), ('duty-free-exports=n',), ('export-administration-act-south-africa=y',), ('adoption-of-the-budget-resolution=y',), ('physician-fee-freeze=n',), ('education-spending=n',), ('Class=democrat',), ('anti-satellite-test-ban=y',), ('aid-to-nicaraguan-contras=y',), ('mx-missile=y',)]

Large itemsets L(2): 21
[('religious-groups-in-schools=y', 'crime=y'), ('religious-groups-in-schools=y', 'export-administration-act-south-africa=y'), ('synfuels-corporation-cutback=n', 'export-administration-act-south-africa=y'), ('adoption-of-the-budget-resolution=y', 'physician-fee-freeze=n'), ('adoption-of-the-budget-resolution=y', 'education-spending=n'), ('adoption-of-the-budget-resolution=y', 'Class=democrat'), ('physi